In [2]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [3]:
robot.getStates()

{'q': array([ 7.40225824e-04, -7.84571029e-01,  2.94816965e-03, -2.37809502e+00,
         6.46072248e-04,  1.56925974e+00,  7.86500187e-01]),
 'dq': array([-3.51572172e-04, -9.69969643e-04, -5.14928865e-05, -1.10261858e-03,
         3.57342407e-04, -9.75935789e-04,  4.61590691e-04]),
 'T': array([-0.08401799, -4.19488668, -0.82716978, 23.14731216,  0.699633  ,
         2.37393928, -0.10297176])}

In [4]:
import numpy as np
from FR3Py.robot.model_collision_avoidance import PinocchioModel
pin_robot = PinocchioModel()
joint_lb = np.array([-2.3093, -1.5133, -2.4937, -2.7478, -2.48, 0.8521, -2.6895])
joint_ub = np.array([2.3093, 1.5133, 2.4937, -0.4461, 2.48, 4.2094, 2.6895])
q_nominal = np.array([0.7237753, 0.80070632, 0.14942601, -1.71996101, -0.16544614,
         2.48563189,  1.75224705, 0.025, 0.025])
pin_info = pin_robot.getInfo(q_nominal, np.zeros_like(q_nominal))
P_EE_desired = pin_info['P_EE']
print("P_EE_desired:", P_EE_desired)
R_EE_desired = np.array([[1, 0, 0],
                        [0, -1, 0],
                        [0, 0, -1]])


P_EE_desired: [0.42722967 0.50070682 0.02100275]


In [5]:
q_initial = np.array([ 1.99664562e-04, -7.83964837e-01,  3.22413486e-03, -2.38147407e+00,
        -7.03871748e-04,  1.57095859e+00,  7.86564759e-01, 0.025, 0.025])
pin_info = pin_robot.getInfo(q_initial, np.zeros_like(q_initial))
P_EE_initial = pin_info['P_EE']
print("P_EE_initial:", P_EE_initial)

P_EE_initial: [0.30374356 0.00092832 0.47456965]


In [33]:
from cores.utils.trajectory_utils import TrapezoidalTrajectory
# P_EE_1 = np.array([0.42722967, 0.50070682, 0.47456965])
# via_points = np.array([P_EE_initial, P_EE_1, P_EE_desired])
# target_time = np.array([0, 8, 15])

via_points = np.array([P_EE_initial, P_EE_desired])
target_time = np.array([0, 15])
traj = TrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=0.01)

In [34]:
import time
from cores.utils.control_utils import get_torque_to_track_traj_const_ori

T = 20
n_joints = 7
q_bar = 0.5*(joint_ub + joint_lb)
# Kp = np.diag([20,20,20,100,100,100])
# Kd = np.diag([20,20,20,100,100,100])

Kp_task = np.diag([20,20,20,50,50,50])
Kd_task = np.diag([20,20,20,50,50,50])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    Minv = pin_info["Minv"][0:n_joints,0:n_joints]  # shape (7,7)
    M = pin_info["M"][0:n_joints,0:n_joints]  # shape (7,7)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint + nle - G
    u = np.clip(u_nominal, -10, 10)
    print(u)
    print(robot_info['T'])
    print("###########")
    robot.setCommands(u)






[ 1.0513823  -0.23031826  1.13476275 -0.50751485  0.05223427 -0.06048215
 -0.02023549]
[-0.08401799 -4.22265768 -0.39473569 23.07193375  0.64103925  2.28787971
  0.06182316]
###########
[ 1.0269145  -0.29435573  1.11445668 -0.47269575  0.05100288 -0.0548513
 -0.02030082]
[-0.06021428 -4.22265768 -0.47011411 23.09573746  0.64103925  2.28787971
  0.06823185]
###########
[ 1.07507859 -0.34740431  1.17016946 -0.45183716  0.0531564  -0.05166826
 -0.02044932]
[-0.08401799 -4.19488668 -0.39473569 23.14731216  0.64103925  2.2942884
  0.07464054]
###########
[ 1.10773983 -0.28147157  1.19897745 -0.51157566  0.05685658 -0.05964529
 -0.01988625]
[-0.03641057 -4.22265768 -0.3669647  23.11954117  0.63463056  2.2942884
  0.10119084]
###########
[ 1.05358455 -0.22736619  1.13687065 -0.52060968  0.0536649  -0.06040802
 -0.0183763 ]
[-0.11178899 -4.17108297 -0.4185394  23.11954117  0.63463056  2.2942884
  0.10759953]
###########
[ 1.05594475 -0.19933637  1.13687823 -0.53781057  0.05231104 -0.06357755
 